# Tendency

**Tendency** là một đặc trưng biểu diễn xu hướng thay đổi của nhịp tim thai nhi theo thời gian. Giá trị của Tendency gồm `-1`, `0` và `1`, lần lượt tương ứng với các trạng thái **giảm**, **không đổi**, và **tăng**.

- Tendency là biến phân loại có thứ tự (Ordinal Categorical Feature), vì các giá trị mang ý nghĩa thứ tự: `giảm < không đổi < tăng`.
- Trong tiền xử lý, thay vì dùng **one-hot encoding** (mất thông tin thứ tự), ta giữ nguyên các giá trị `-1`, `0`, `1`.
- Để đồng nhất thang đo giữa các đặc trưng, Tendency được **chuẩn hóa** cùng với các đặc trưng liên tục khác bằng:
  - `StandardScaler`
  - hoặc `MinMaxScaler`
- Việc chuẩn hóa giúp mô hình đánh giá đúng mức độ ảnh hưởng của từng đặc trưng, không bị lệch bởi đơn vị đo khác nhau.


# Nhãn có thứ tự

## Khoảng cách trong KNN

- Nhãn `NSP` là dữ liệu có thứ tự:  
  `1: Bình thường`, `2: Nghi ngờ`, `3: Bệnh lý`.
- Nên sử dụng:
  - **Euclidean distance**: phù hợp nếu các đặc trưng cùng đơn vị.
  - **Manhattan distance**: tốt hơn khi dữ liệu có phân bố không đều hoặc nhiều giá trị cực đoan.

## Hạn chế của Accuracy

- **Accuracy không phân biệt lỗi nặng hay nhẹ**:
  - Dự đoán sai `1 -> 3` nguy hiểm hơn `1 -> 2`, nhưng Accuracy coi như nhau.

### Nên dùng các chỉ số sau:

#### 1. MAE (Mean Absolute Error)
- Đo khoảng cách sai lệch giữa nhãn thực và dự đoán.
- Ví dụ:
  - `Dự đoán 1 -> 3`: MAE = |1 - 3| = 2 (lỗi nặng)
  - `Dự đoán 1 -> 2`: MAE = |1 - 2| = 1 (lỗi nhẹ)
- **MAE càng nhỏ càng tốt.**

#### 2. Weighted F1-Score
- Đánh giá riêng từng lớp, có trọng số theo tỷ lệ dữ liệu.
- Phát hiện tình trạng mô hình bỏ qua lớp thiểu số.

### Ví dụ:

| Mô hình   | Accuracy | MAE  |
|-----------|----------|------|
| A         | 90%      | 0.5  |
| B         | 90%      | 1.2  |

=> Mô hình A tốt hơn vì ít mắc lỗi nghiêm trọng hơn.

### Tóm lại:

- **MAE**: Tránh lỗi chẩn đoán nguy hiểm.
- **F1-Score**: Đảm bảo phát hiện đủ các mức độ.

> Kết hợp cả hai chỉ số để đánh giá đúng chất lượng mô hình.



# Softmax vs Ordinal Regression

- **Softmax** (Multinomial Logistic Regression):  
  => Coi các lớp là không có thứ tự => Không phù hợp với NSP.
- Nhược điểm:
  - Không phân biệt lỗi `1 -> 2` với `1 -> 3`.
- **Giải pháp**: Dùng **Ordinal Logistic Regression** để tận dụng thông tin thứ tự giữa các lớp.


# Mô hình SVM

## 1. Giới thiệu về SVM
- SVM (Support Vector Machine): mô hình học có giám sát, dùng cho phân loại và hồi quy.
- Ý tưởng: tìm **siêu phẳng (hyperplane)** phân chia hai lớp sao cho **khoảng cách (margin)** tới các điểm gần nhất là lớn nhất.

## 2. Margin (Khoảng cách)
- Là khoảng cách từ siêu phẳng đến các điểm gần nhất.
- Các điểm này gọi là **support vectors**.
- Mục tiêu: **tối đa hóa margin** để tránh overfitting.

## 3. SVM tuyến tính

### 3.1 Mô tả dữ liệu
- Dữ liệu huấn luyện: (xᵢ, yᵢ) với xᵢ ∈ ℝᵈ, yᵢ ∈ {-1, 1}.

### 3.2 Mô hình siêu phẳng
- Phương trình: `wᵗx + b = 0`
  - `w`: vector vuông góc với siêu phẳng
  - `b`: độ dịch chuyển

### 3.3 Ràng buộc phân loại
- Mỗi điểm phải thỏa: `yᵢ(wᵗxᵢ + b) ≥ 1`

### 3.4 Bài toán tối ưu
- Tối ưu: `min (1/2)||w||²`
- Với ràng buộc: `yᵢ(wᵗxᵢ + b) ≥ 1 ∀ i`

### 3.5 Giải bằng Lagrangian
- L(w, b, α) = (1/2)||w||² − Σαᵢ[yᵢ(wᵗxᵢ + b) − 1]
- αᵢ ≥ 0: hệ số Lagrange.
- Giải bài toán đối ngẫu (dual).

## 4. Soft Margin SVM
- Khi dữ liệu không phân cách được hoàn toàn, thêm biến **slack ξᵢ ≥ 0**:
  - Ràng buộc: `yᵢ(wᵗxᵢ + b) ≥ 1 − ξᵢ`
- Bài toán tối ưu:
  - `min (1/2)||w||² + C Σξᵢ`
  - `C`: siêu tham số cân bằng giữa margin lớn và ít vi phạm.

## 5. Kernel Trick (phi tuyến)
- Ánh xạ dữ liệu lên không gian đặc trưng cao hơn.
- Không cần ánh xạ tường minh nhờ **kernel function**:
  - `K(x, x') = φ(x)ᵗφ(x')`

### 5.1 Các hàm kernel phổ biến
- **Linear**: `K(x, x') = xᵗx'`
- **Polynomial**: `K(x, x') = (xᵗx' + c)^d`
- **Gaussian (RBF)**: `K(x, x') = exp(-||x − x'||² / 2σ²)`

## 6. Tóm tắt
- SVM mạnh nhờ tối ưu khoảng cách.
- **Kernel trick** xử lý dữ liệu phi tuyến hiệu quả.
- **Soft margin** cho phép mô hình chịu được dữ liệu không hoàn hảo.
